# HOMEWORK 4, BY SHRIKANTH SUBRAMANIAN

In [1]:
#Libraries used for data analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import itertools
import sklearn.linear_model as skl_lm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_squared_error

In [2]:
#Reading data into pandas dataframes for analysis
train1 = pd.read_csv(r'C:\Users\shrik\Downloads\HW4_data\training.csv')
train2 = pd.read_csv(r'C:\Users\shrik\Downloads\HW4_data\formula_training.csv')
final_test_1 = pd.read_csv(r'C:\Users\shrik\Downloads\HW4_data\test.csv')
final_test_2 = pd.read_csv(r'C:\Users\shrik\Downloads\HW4_data\formula_test.csv')



FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\shrik\\Downloads\\HW4_data\\training.csv'

In [ ]:
#Checking the dimensions of the data
print(len(train1))
train1.head()


In [ ]:
#Verifying whether the number of rows is same in both data sheets
print(len(train2))
train2.head()

In [ ]:
#Here we define our target variable
y = train1['critical_temp']
X = train1.drop('critical_temp', 1)


In [ ]:
#Let us check to see how the data looks (10 random graphs chosen, might not be able to form meaningful conclusions).
count = 0
for i in X:
    while(count<10):
        print("Plot for {} vs. Critical Temperature".format(i))
        plt.scatter(X[i], y)
        plt.show()
        count += 1
    continue

Since there are so many plots, it is very difficult to gauge the correlation by eye. So we are now going to use a correlation matrix.

In [ ]:
#Correlation matrix is found
correlation_matrix = train1.corr()
correlation_matrix

In [ ]:
#We remove the critical_temp row from correlation matrix to avoid duplication.
correlation_matrix = correlation_matrix.drop('critical_temp',0) 
correlation_matrix

In [ ]:
# We can say there is a strong correlation if the value is between |0.5|<x<|1.0| where x is the correlation factor.
corr = correlation_matrix['critical_temp'].sort_values(ascending=False)
corr.columns = ['Columns', 'Corr_index']
negcorr = corr[(corr<=-0.5) & (corr>=-1)]
poscorr = corr[(corr>=0.5) & (corr<=1) ]
print(poscorr)    
print(negcorr)

In [ ]:
columns1 = ['wtd_std_ThermalConductivity', 
 'range_ThermalConductivity', 
 'range_atomic_radius', 
 'std_ThermalConductivity',        
 'wtd_entropy_atomic_mass',        
 'range_fie',                      
 'wtd_entropy_atomic_radius',      
 'number_of_elements',             
 'entropy_Valence',                
 'wtd_entropy_Valence',            
 'wtd_std_atomic_radius',          
 'wtd_std_fie',                    
 'entropy_fie',                    
 'entropy_atomic_radius',          
 'wtd_entropy_FusionHeat',         
 'std_atomic_radius',              
 'std_fie',                        
 'entropy_FusionHeat',             
 'entropy_atomic_mass',            
 'range_atomic_mass',              
 'wtd_gmean_Density',   
 'gmean_Density',       
 'gmean_Valence',       
 'mean_Valence',        
 'wtd_gmean_Valence',   
 'wtd_mean_Valence']

In [ ]:
#Let us check to see how the chemical data looks (10 random graphs chosen, might not be able to form meaningful conclusions).
count = 0
for i in train2:
    while(count < 10):
        print("Plot for {} vs. Critical Temperature".format(i))
        plt.scatter(train2[i], y)
        plt.show()
        count += 1
    continue

In [ ]:
#Correlation data for dataset 2
correlation_matrix = train2.corrwith(y, axis=0, drop=False, method='pearson')
negcorr = correlation_matrix[(correlation_matrix<=-0.5) & (correlation_matrix>=-1)]
poscorr = correlation_matrix[(correlation_matrix>=0.5) & (correlation_matrix<=1) ]

print(poscorr)    
print(negcorr)

Chemical formula is weakly correlated for all elements except O, Cu, Ba

In [ ]:
columns2 = ['O', 'Cu', 'Ba']


In [ ]:
X = train1[['wtd_std_ThermalConductivity', 
 'range_ThermalConductivity', 
 'range_atomic_radius', 
 'std_ThermalConductivity',        
 'wtd_entropy_atomic_mass',        
 'range_fie',                      
 'wtd_entropy_atomic_radius',      
 'number_of_elements',             
 'entropy_Valence',                
 'wtd_entropy_Valence',            
 'wtd_std_atomic_radius',          
 'wtd_std_fie',                    
 'entropy_fie',                    
 'entropy_atomic_radius',          
 'wtd_entropy_FusionHeat',         
 'std_atomic_radius',              
 'std_fie',                        
 'entropy_FusionHeat',             
 'entropy_atomic_mass',            
 'range_atomic_mass',              
 'wtd_gmean_Density',   
 'gmean_Density',       
 'gmean_Valence',       
 'mean_Valence',        
 'wtd_gmean_Valence',   
 'wtd_mean_Valence']]

In [ ]:
X_temp = train2[['O', 'Cu', 'Ba']]


In [ ]:
#The training dataset is prepared
X = pd.concat([X, X_temp], axis=1)



In [ ]:
#All structural changes made to the training dataset has been made to reflect in testing dataset
X_final_test = final_test_1[['wtd_std_ThermalConductivity', 'range_ThermalConductivity', 'range_atomic_radius', 'std_ThermalConductivity','wtd_entropy_atomic_mass','range_fie',                      
'wtd_entropy_atomic_radius','number_of_elements', 'entropy_Valence', 'wtd_entropy_Valence','wtd_std_atomic_radius','wtd_std_fie',                    
'entropy_fie', 'entropy_atomic_radius','wtd_entropy_FusionHeat', 'std_atomic_radius', 'std_fie', 'entropy_FusionHeat', 'entropy_atomic_mass',            
'range_atomic_mass', 'wtd_gmean_Density','gmean_Density','gmean_Valence','mean_Valence','wtd_gmean_Valence', 'wtd_mean_Valence']]
X_final_test_temp = final_test_2[['O', 'Cu', 'Ba']]
X_final_test = pd.concat([X_final_test, X_final_test_temp], axis=1)
X_final_test


In [ ]:
scaler = StandardScaler().fit(X)

alphas = 10**np.linspace(10,-2,100)
ridge = skl_lm.Ridge()
coefs = []

for a in alphas:
    ridge.set_params(alpha=a)
    ridge.fit(scaler.transform(X), y)
    coefs.append(ridge.coef_)

In [ ]:
#np.amin(mse_per_component)

In [ ]:
len(coefs)

In [ ]:
#Ridge Regression
def Ridge_Regression(X, y, alpha):
    scaler = StandardScaler()
    ridge = skl_lm.Ridge()
    ridge.set_params(alpha=alpha)
    ridge.fit(scaler.fit_transform(X), y)
    return ridge.coef_

In [ ]:
Ridge_Regression(X, y, alpha=705)

In [ ]:
#Training data is being split into train and test to verify accuracy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

ridgecv = skl_lm.RidgeCV(alphas=alphas, scoring='neg_mean_squared_error')
ridgecv.fit(scaler.fit_transform(X_train), y_train)

In [ ]:
ridgecv.alpha_

In [ ]:
optimal_ridge = skl_lm.Ridge()
optimal_ridge.set_params(alpha=ridgecv.alpha_)
optimal_ridge.fit(scaler.fit_transform(X_train), y_train)

In [ ]:
#MSE of 382 is lower than the other MSE for different models
mean_squared_error(y_test, optimal_ridge.predict(scaler.fit_transform(X_test)))

In [ ]:
pd.Series(optimal_ridge.coef_.flatten(), index=X.columns)

In [ ]:
#Lasso regression
lassocv = skl_lm.LassoCV(alphas = alphas, max_iter=10000)
lassocv.fit(scaler.fit_transform(X_train), y_train.values.ravel())

In [ ]:
lassocv.alpha_

In [ ]:
#2nd lowest MSE found
lasso = skl_lm.Lasso()
lasso.set_params(alpha = lassocv.alpha_)
lasso.fit(scaler.fit_transform(X_train), y_train.values.ravel())
mean_squared_error(y_test, lasso.predict(scaler.fit_transform(X_test)))

In [ ]:
pd.Series(lasso.coef_.flatten(), index = X.columns)

In [ ]:
pca = PCA()
X_train_reduced = pca.fit_transform(scaler.fit_transform(X_train))

# We will use OLS to fit our M-dimensional data, derived from PCA
regr = skl_lm.LinearRegression()

# 10-fold CV, with shuffle
kf_10 = KFold(n_splits=10, shuffle=True, random_state=1)
mse = []

for i in np.arange(1, 28):
    score = -1*cross_val_score(regr, X_train_reduced[:,:i], y_train, cv=kf_10, scoring='neg_mean_squared_error').mean()
    mse.append(score)
    
mse_per_component = pd.Series(np.array(mse).flatten(), index = np.arange(1,28))
print(mse_per_component)

In [ ]:
np.amin(mse_per_component)

In [ ]:
#Principal Component Analysis
# optimal_pca = PCA(n_components=18)
X_test_reduced = pca.transform(scaler.fit_transform(X_test))[:, :18]

# train OLS model on PCA-reduced training data
pca_regr = skl_lm.LinearRegression()
pca_regr.fit(X_train_reduced[:,:18], y_train.values.ravel())

# Make predictions with our model and calculate MSE
y_pred = pca_regr.predict(X_test_reduced)
mean_squared_error(y_test.values.ravel(), y_pred)

In [ ]:
#PLS Regression
pls = PLSRegression()
pls.fit(scaler.fit_transform(X_train), y_train)

mean_squared_error(y_test, pls.predict(scaler.fit_transform(X_test)))

In [ ]:
optimal_ridge.fit(scaler.fit_transform(X), y)
prediction_final = optimal_ridge.predict(scaler.fit_transform(X_final_test))

In [ ]:
#Exporting CSV
#We are choosing Ridge Regression due to its lower MSE
res = pd.DataFrame(prediction_final)
res.index = X_final_test.index # its important for comparison
res.columns = ["critical_temp"]
res.to_csv("prediction_results.csv") 